##### Configurando as variáveis de ambiente

In [1]:
# Configurando as variáveis de ambiente

import os 
os.environ["SPARK_HOME"] = "C:\spark\spark-3.1.3-bin-hadoop2.7"

##### Iniciando uma sessão local

In [2]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.master('local[*]').getOrCreate()

##### Tornar o pyspark "importável" e Importando bibliotecas

In [3]:
# Tornar o pyspark "importável"
import findspark
findspark.init()

# Importando bibliotecas
import pandas as pd
from pyspark.sql.functions import col, to_date, hour, year, month, to_timestamp, dayofmonth
from pyspark.sql.types import IntegerType

### Leitura dos dados 
#### Foi realizado a leitura de uma planilha Excel, onde cada tabela estava em uma planilha diferente.

In [4]:
list = ['dCopo', 'fPedidos', 'dCidade', 'dEntrega', 'dPagamento', 'dAcompanhamento', 'dCobertura']
path = 'D:\Acai_power'

df_copo = spark.read.option("delimiter", ";").option("header", "true").csv(f'{path}\{list[0]}.csv')

df_pedidos = spark.read.option("delimiter", ";").option("header", "true").csv(f'{path}\{list[1]}.csv')

df_cidade = spark.read.option("delimiter", ";").option("header", "true").csv(f'{path}\{list[2]}.csv')

df_entrega = spark.read.option("delimiter", ";").option("header", "true").csv(f'{path}\{list[3]}.csv')

df_pagamento = spark.read.option("delimiter", ";").option("header", "true").csv(f'{path}\{list[4]}.csv')

df_acompanhamento = spark.read.option("delimiter", ";").option("header", "true").csv(f'{path}\{list[5]}.csv')

df_cobertura = spark.read.option("delimiter", ";").option("header", "true").csv(f'{path}\{list[6]}.csv')

AnalysisException: Path does not exist: file:/D:/Acai_power/dCopo.csv

In [ ]:
# Tratamento no Dataframe pedidos, convertendo a coluna DataHora para datetime

df_pedidos = df_pedidos.withColumn('DataHora', to_timestamp('DataHora'))

### Join entre os Dataframes
#### Após a leitura, foi realizado join entre o Dataframe Pedidos com os demais Dataframes, com o intuito de criar somente um tabelão, retirando os códigos e substituindo pelas informações descritivas.

In [ ]:
# Join com o Dataframe Copo, trazendo informações como o volume do copo e valor do copo
cond = [df_pedidos.Copo == df_copo.IdCopo]
pedidos = df_pedidos.join(df_copo, cond, how='left').drop('IdCopo', 'Copo')

# Join com o Dataframe Cidade, trazendo informações de localização do cliente
cond = [pedidos.Cidade == df_cidade.IdCidade]
pedidos = pedidos.join(df_cidade, cond, how='left').drop('IdCidade', 'Cidade')

# Join com o Dataframe Entrega, trazendo informações sobre o tipo de entrega
cond = [pedidos.Entrega == df_entrega.IdEntrega]
pedidos = pedidos.join(df_entrega, cond, how='left').drop('IdEntrega', 'Entrega')

# Join com o Dataframe Pagamento, trazendo as informações de pagamento do cliente
cond = [pedidos.Pagamento == df_pagamento.IdPagamento]
pedidos = pedidos.join(df_pagamento, cond, how='left').drop('IdPagamento', 'Pagamento')

# Join com o Dataframe Cobertura, trazendo informação da cobertura escolhida pelo cliente
cond = [pedidos.Cobertura == df_cobertura.IdCobertura]
pedidos = pedidos.join(df_cobertura, cond, how='left').drop('IdCobertura', 'Cobertura')


# Join com o Dataframe Acompanhamentos, trazendo informações sobre os acompanhamentos solicitados. Como são 3 acompanhamentos, o processo foi realizado um de cada vez.

# Acompanhamento1
cond = [pedidos.Acomp1 == df_acompanhamento.IdAcompanhamento]
pedidos = pedidos.join(df_acompanhamento, cond, how='left').drop('Acomp1', 'IdAcompanhamento')
pedidos = pedidos.withColumnRenamed('NomAcompanhamento', 'Acompanhamento1')

# Acompanhamento2
cond = [pedidos.Acomp2 == df_acompanhamento.IdAcompanhamento]
pedidos = pedidos.join(df_acompanhamento, cond, how='left').drop('Acomp2', 'IdAcompanhamento')
pedidos = pedidos.withColumnRenamed('NomAcompanhamento', 'Acompanhamento2')

# Acompanhamento3
cond = [pedidos.Acomp3 == df_acompanhamento.IdAcompanhamento]
pedidos = pedidos.join(df_acompanhamento, cond, how='left').drop('Acomp3', 'IdAcompanhamento')
pedidos = pedidos.withColumnRenamed('NomAcompanhamento', 'Acompanhamento3')

### Selecionando colunas, renomeando e adicionando novas. 

In [ ]:
pedidos = pedidos.select(
    col('DataHora')
    ,to_date(col("DataHora"),"yyyy-MM-dd").alias("Data")
    ,hour(col('DataHora')).alias('HoraPedido')
    ,col('IdPedido')
    ,col('CategoriaCopo').alias('TamanhoCopo')
    ,col('VlrPrecoCopo').alias('Valor')
    ,col('NomCidade').alias('Cidade')
    ,col('NomEstado').alias('Estado')
    ,col('NomRegiao').alias('Regiao')
    ,col('NomEntrega').alias('TipoEntrega')
    ,col('NomPagamento').alias('TipoPagamento')
    ,col('NomCobertura').alias('Cobertura')
    ,col('Acompanhamento1')
    ,col('Acompanhamento2')
    ,col('Acompanhamento3')
)

## Análise dos Dados
### Respondendo perguntas

### Análise dos pedidos

##### Pedidos por ano e mês

In [ ]:
# Pedidos por ano e mês
pedidos.groupby(year(col('Data')).alias('Ano'),\
month(col('Data')).alias('Mes'))\
.count()\
.orderBy('ano', 'mes')\
.withColumnRenamed('count', 'Contagem Pedidos')\
.show()

+----+---+----------------+
| Ano|Mes|Contagem Pedidos|
+----+---+----------------+
|2020|  1|            1407|
|2020|  2|            1359|
|2020|  3|            1607|
|2020|  4|            1348|
|2020|  5|            1472|
|2020|  6|            1484|
|2020|  7|            1523|
|2020|  8|            1426|
|2020|  9|            1389|
|2020| 10|            1574|
|2020| 11|            1420|
|2020| 12|            1468|
|2021|  1|            1465|
|2021|  2|            1314|
|2021|  3|            1590|
|2021|  4|            1380|
|2021|  5|            1480|
|2021|  6|            1346|
|2021|  7|            1542|
|2021|  8|            1451|
+----+---+----------------+
only showing top 20 rows



##### Pedidos por dia do mês

In [ ]:
pedidos.groupBy(dayofmonth('Data')\
    .alias('Dia do Mes'))\
    .count()\
    .withColumnRenamed('count', 'Contagem Pedidos')\
    .orderBy('Contagem Pedidos', ascending = False)\
    .show()

+----------+----------------+
|Dia do Mes|Contagem Pedidos|
+----------+----------------+
|         1|            3169|
|         3|            2411|
|         2|            2400|
|         8|            2375|
|        25|            2340|
|        18|            2330|
|        12|            2320|
|        21|            2315|
|        17|            2313|
|        15|            2293|
|        11|            2282|
|        26|            2275|
|         5|            2266|
|        16|            2262|
|        14|            2259|
|        13|            2257|
|        27|            2247|
|         7|            2246|
|        24|            2243|
|        23|            2242|
+----------+----------------+
only showing top 20 rows



##### Horário em que foram realizados mais pedidos

In [ ]:
pedidos.groupby(col('HoraPedido'))\
    .count()\
    .withColumnRenamed('count', 'Contagem Pedidos')\
    .orderBy('Contagem Pedidos', ascending= False)\
    .show()

+----------+----------------+
|HoraPedido|Contagem Pedidos|
+----------+----------------+
|        10|            5930|
|        13|            5875|
|        18|            5867|
|        14|            5852|
|        11|            5841|
|        20|            5840|
|        16|            5839|
|        17|            5834|
|        19|            5798|
|        12|            5771|
|        15|            5764|
|         9|            5698|
|        21|              91|
+----------+----------------+



##### Pedidos por cidade

In [ ]:
pedidos.groupBy('Cidade')\
    .count()\
    .withColumnRenamed('count', 'Contagem Pedidos')\
    .orderBy('Contagem Pedidos', ascending= False)\
    .show()

+-------------------+----------------+
|             Cidade|Contagem Pedidos|
+-------------------+----------------+
|         Petrópolis|            4232|
|           Blumenau|            4206|
|             Itajaí|            4190|
|     Rio de Janeiro|            4186|
|São José dos Campos|            4163|
|          Fortaleza|            4137|
|      Florianópolis|            4132|
|          Joinville|            4120|
|        Nova Iguaçu|            4109|
|  Juazeiro do Norte|            4105|
|             Sobral|            4102|
|    Duque de Caxias|            4086|
|           Caieiras|            4076|
|          Nilópolis|            4072|
|         Jaguariúna|            4070|
|   Campos do Jordão|            4010|
|          São Paulo|            4004|
+-------------------+----------------+



##### Pedidos por estado

In [ ]:
pedidos.groupBy('Estado')\
    .count()\
    .withColumnRenamed('count', 'Contagem Pedidos')\
    .orderBy('Contagem Pedidos', ascending= False)\
    .show()

+--------------+----------------+
|        Estado|Contagem Pedidos|
+--------------+----------------+
|Rio de Janeiro|           20685|
|     São Paulo|           20323|
|Santa Catarina|           16648|
|         Ceará|           12344|
+--------------+----------------+



##### Pedidos por região

In [ ]:
pedidos.groupBy('Regiao')\
    .count()\
    .withColumnRenamed('count', 'Contagem Pedidos')\
    .orderBy('Contagem Pedidos', ascending= False)\
    .show()

+--------+----------------+
|  Regiao|Contagem Pedidos|
+--------+----------------+
| Sudeste|           41008|
|     Sul|           16648|
|Nordeste|           12344|
+--------+----------------+



##### Pedidos por tipo de consumo (no local ou delivery)

In [ ]:
pedidos.groupBy('TipoEntrega')\
    .count()\
    .withColumnRenamed('count', 'Contagem Pedidos')\
    .orderBy('Contagem Pedidos', ascending= False)\
    .show()

+----------------+----------------+
|     TipoEntrega|Contagem Pedidos|
+----------------+----------------+
|Consumo no local|           35099|
|        Delivery|           34901|
+----------------+----------------+



##### Pedidos por forma de pagamento

In [ ]:
pedidos.groupBy('TipoPagamento')\
    .count()\
    .withColumnRenamed('count', 'Contagem Pedidos')\
    .orderBy('Contagem Pedidos', ascending= False)\
    .show()

+-------------+----------------+
|TipoPagamento|Contagem Pedidos|
+-------------+----------------+
|     Dinheiro|           17649|
|       Débito|           17548|
|      Crédito|           17459|
|          Pix|           17344|
+-------------+----------------+



##### Coberturas mais pedidas

In [ ]:
pedidos.groupBy('Cobertura')\
    .count()\
    .withColumnRenamed('count', 'Contagem Pedidos')\
    .orderBy('Contagem Pedidos', ascending= False)\
    .show()

+------------+----------------+
|   Cobertura|Contagem Pedidos|
+------------+----------------+
|       Menta|            7212|
|    Maracujá|            7111|
|Tutti-frutti|            7085|
|   Chocolate|            7041|
|    Blue Ice|            6999|
|       Limão|            6980|
|     Morango|            6919|
|         Uva|            6902|
|    Chiclete|            6892|
|     Abacaxi|            6859|
+------------+----------------+



#### Analisando os acompanhamentos pedidos
##### Para isso, foram criados 3 Dataframes com o Id do pedido e também a descrição do acompanhamento, e logo após o Union para a análise ser realizada.

In [ ]:
# Criando 3 Dataframes com o IdPedido e o acompanhamento 1, 2 e 3
acompanhamento1 = pedidos.select(col('Acompanhamento1').alias('Acompanhamento'), col('IdPedido'))
acompanhamento2 = pedidos.select(col('Acompanhamento2').alias('Acompanhamento'), col('IdPedido'))
acompanhamento3 = pedidos.select(col('Acompanhamento3').alias('Acompanhamento'), col('IdPedido'))

In [ ]:
# Realizando o Union dos 3 Dataframes
acompanhamentos = acompanhamento1.unionAll(acompanhamento2).unionAll(acompanhamento3)

##### Acompanhamentos mais pedidos

In [ ]:
acompanhamentos.groupBy('Acompanhamento')\
    .count()\
    .withColumnRenamed('count', 'Contagem Pedidos')\
    .orderBy('Contagem Pedidos', ascending= False)\
    .show()

+--------------------+----------------+
|      Acompanhamento|Contagem Pedidos|
+--------------------+----------------+
|            Castanha|           19225|
|      Floco de arroz|           19208|
|         Leite Ninho|           19189|
|Confete de chocolate|           19185|
|             Granola|           19145|
|              Paçoca|           19099|
|              Jujuba|           19059|
|       Tubo de wafer|           19056|
|       Nescau cereal|           19005|
|           Granulado|           18946|
|           Sucrilhos|           18883|
+--------------------+----------------+



##### Pedidos por tamanho do copo (em ml)

In [ ]:
pedidos.groupBy('TamanhoCopo')\
    .count()\
    .withColumnRenamed('count', 'Contagem Pedidos')\
    .orderBy('Contagem Pedidos', ascending= False)\
    .show()

+-----------+----------------+
|TamanhoCopo|Contagem Pedidos|
+-----------+----------------+
|     1000ml|           14134|
|      500ml|           13986|
|      200ml|           13975|
|      700ml|           13961|
|      300ml|           13944|
+-----------+----------------+



##### Pedidos por tamanho do copo (em ml) em cada cidade

In [ ]:
pedidos.groupBy('Cidade', 'TamanhoCopo')\
    .count()\
    .withColumnRenamed('count', 'Contagem Pedidos')\
    .orderBy('Contagem Pedidos', ascending= False)\
    .show()

+-------------------+-----------+----------------+
|             Cidade|TamanhoCopo|Contagem Pedidos|
+-------------------+-----------+----------------+
|           Blumenau|      700ml|             897|
|         Petrópolis|      500ml|             891|
|     Rio de Janeiro|      700ml|             887|
|São José dos Campos|      200ml|             886|
|          Joinville|     1000ml|             874|
|         Jaguariúna|      700ml|             870|
|São José dos Campos|      700ml|             863|
|      Florianópolis|      500ml|             861|
|           Blumenau|     1000ml|             861|
|          Fortaleza|      500ml|             860|
|          Fortaleza|     1000ml|             860|
|         Petrópolis|      200ml|             855|
|             Itajaí|      300ml|             854|
|  Juazeiro do Norte|      200ml|             854|
|        Nova Iguaçu|     1000ml|             851|
|    Duque de Caxias|      300ml|             850|
|             Itajaí|     1000m

##### Pedidos por horário quando o consumo foi no local

In [ ]:
# Consumo no local por hora
pedidos.groupBy('HoraPedido', 'TipoEntrega')\
    .count()\
    .withColumnRenamed('count', 'Contagem Pedidos')\
    .filter(col('TipoEntrega') == 'Consumo no local')\
    .orderBy('Contagem Pedidos', ascending= False)\
    .show()

+----------+----------------+----------------+
|HoraPedido|     TipoEntrega|Contagem Pedidos|
+----------+----------------+----------------+
|        13|Consumo no local|            3000|
|        10|Consumo no local|            2977|
|        17|Consumo no local|            2958|
|        16|Consumo no local|            2954|
|        20|Consumo no local|            2920|
|        19|Consumo no local|            2912|
|        18|Consumo no local|            2910|
|        15|Consumo no local|            2908|
|        11|Consumo no local|            2898|
|         9|Consumo no local|            2893|
|        12|Consumo no local|            2863|
|        14|Consumo no local|            2860|
|        21|Consumo no local|              46|
+----------+----------------+----------------+



##### Pedidos por horário quando foi solicitado Delivery

In [ ]:
# Delivery por hora
pedidos.groupBy('HoraPedido', 'TipoEntrega')\
    .count()\
    .withColumnRenamed('count', 'Contagem Pedidos')\
    .filter(col('TipoEntrega') == 'Delivery')\
    .orderBy('Contagem Pedidos', ascending= False)\
    .show()

+----------+-----------+----------------+
|HoraPedido|TipoEntrega|Contagem Pedidos|
+----------+-----------+----------------+
|        14|   Delivery|            2992|
|        18|   Delivery|            2957|
|        10|   Delivery|            2953|
|        11|   Delivery|            2943|
|        20|   Delivery|            2920|
|        12|   Delivery|            2908|
|        19|   Delivery|            2886|
|        16|   Delivery|            2885|
|        17|   Delivery|            2876|
|        13|   Delivery|            2875|
|        15|   Delivery|            2856|
|         9|   Delivery|            2805|
|        21|   Delivery|              45|
+----------+-----------+----------------+



##### Pedidos por mês quando o consumo foi no local

In [ ]:
# Consumo no local por mes
pedidos.groupBy(month('Data').alias('Mes'), 'TipoEntrega')\
    .count()\
    .withColumnRenamed('count', 'Contagem Pedidos')\
    .filter(col('TipoEntrega') == 'Consumo no local')\
    .orderBy('Contagem Pedidos', ascending= False)\
    .show()

+---+----------------+----------------+
|Mes|     TipoEntrega|Contagem Pedidos|
+---+----------------+----------------+
|  3|Consumo no local|            3185|
|  7|Consumo no local|            3139|
| 10|Consumo no local|            3057|
|  5|Consumo no local|            3056|
| 12|Consumo no local|            2991|
|  9|Consumo no local|            2903|
|  1|Consumo no local|            2863|
|  8|Consumo no local|            2850|
| 11|Consumo no local|            2815|
|  6|Consumo no local|            2810|
|  4|Consumo no local|            2769|
|  2|Consumo no local|            2661|
+---+----------------+----------------+



##### Pedidos por mês quando foi solicitado Delivery

In [ ]:
# Consumo no local por mes
pedidos.groupBy(month('Data').alias('Mes'), 'TipoEntrega')\
    .count()\
    .withColumnRenamed('count', 'Contagem Pedidos')\
    .filter(col('TipoEntrega') == 'Delivery')\
    .orderBy('Contagem Pedidos', ascending= False)\
    .show()

+---+-----------+----------------+
|Mes|TipoEntrega|Contagem Pedidos|
+---+-----------+----------------+
|  3|   Delivery|            3120|
| 10|   Delivery|            3024|
|  7|   Delivery|            3004|
| 12|   Delivery|            2979|
|  5|   Delivery|            2953|
|  6|   Delivery|            2921|
|  1|   Delivery|            2909|
|  8|   Delivery|            2908|
| 11|   Delivery|            2905|
|  4|   Delivery|            2766|
|  9|   Delivery|            2732|
|  2|   Delivery|            2680|
+---+-----------+----------------+



##### Pedidos por dia quando o consumo foi no local

In [ ]:
# Consumo no local por mes
pedidos.groupBy(dayofmonth('Data').alias('Dia'), 'TipoEntrega')\
    .count()\
    .withColumnRenamed('count', 'Contagem Pedidos')\
    .filter(col('TipoEntrega') == 'Consumo no local')\
    .orderBy('Contagem Pedidos', ascending= False)\
    .show()

+---+----------------+----------------+
|Dia|     TipoEntrega|Contagem Pedidos|
+---+----------------+----------------+
|  1|Consumo no local|            1549|
|  3|Consumo no local|            1237|
| 15|Consumo no local|            1180|
|  2|Consumo no local|            1174|
| 23|Consumo no local|            1168|
| 18|Consumo no local|            1166|
| 25|Consumo no local|            1160|
| 14|Consumo no local|            1159|
|  8|Consumo no local|            1158|
| 27|Consumo no local|            1156|
| 28|Consumo no local|            1154|
|  6|Consumo no local|            1150|
| 12|Consumo no local|            1147|
| 26|Consumo no local|            1142|
| 11|Consumo no local|            1140|
|  7|Consumo no local|            1132|
| 10|Consumo no local|            1131|
|  9|Consumo no local|            1126|
| 13|Consumo no local|            1125|
| 21|Consumo no local|            1125|
+---+----------------+----------------+
only showing top 20 rows



##### Pedidos por dia quando foi solicitado Delivery

In [ ]:
pedidos.groupBy(dayofmonth('Data').alias('Dia'), 'TipoEntrega')\
    .count()\
    .withColumnRenamed('count', 'Contagem Pedidos')\
    .filter(col('TipoEntrega') == 'Delivery')\
    .orderBy('Contagem Pedidos', ascending= False)\
    .show()

+---+-----------+----------------+
|Dia|TipoEntrega|Contagem Pedidos|
+---+-----------+----------------+
|  1|   Delivery|            1620|
|  2|   Delivery|            1226|
|  8|   Delivery|            1217|
| 21|   Delivery|            1190|
| 17|   Delivery|            1189|
| 25|   Delivery|            1180|
|  3|   Delivery|            1174|
| 12|   Delivery|            1173|
| 18|   Delivery|            1164|
| 16|   Delivery|            1162|
|  4|   Delivery|            1159|
|  5|   Delivery|            1154|
| 24|   Delivery|            1145|
| 11|   Delivery|            1142|
| 20|   Delivery|            1134|
| 26|   Delivery|            1133|
| 13|   Delivery|            1132|
|  7|   Delivery|            1114|
| 15|   Delivery|            1113|
| 22|   Delivery|            1103|
+---+-----------+----------------+
only showing top 20 rows



##### Pedidos por mês e por cidade

In [ ]:
pedidos.groupBy(month('Data').alias('Mes'), 'Cidade')\
    .count()\
    .withColumnRenamed('count', 'Contagem Pedidos')\
    .orderBy('Contagem Pedidos', ascending= False)\
    .show()

+---+-------------------+----------------+
|Mes|             Cidade|Contagem Pedidos|
+---+-------------------+----------------+
|  7|     Rio de Janeiro|             404|
|  3|             Itajaí|             401|
|  3|   Campos do Jordão|             391|
|  7|          Joinville|             391|
| 10|        Nova Iguaçu|             388|
|  3|        Nova Iguaçu|             387|
|  3|São José dos Campos|             386|
|  7|           Blumenau|             385|
|  1|São José dos Campos|             384|
|  3|          Fortaleza|             384|
| 10|          Fortaleza|             382|
|  7|          Nilópolis|             381|
|  5|          Nilópolis|             381|
| 12|         Petrópolis|             380|
| 10|  Juazeiro do Norte|             380|
| 12|          Joinville|             380|
|  3|    Duque de Caxias|             377|
| 10|           Blumenau|             377|
|  3|           Blumenau|             376|
|  5|    Duque de Caxias|             376|
+---+------

### Faturamento

#### Será analisado o faturamento pelos descritivos contidos na base

In [ ]:
# Convertendo a coluna valor para numeric
pedidos = pedidos.withColumn("Valor", col("Valor").cast(IntegerType()))

##### Faturamento por horário do dia

In [ ]:
pedidos.groupBy('HoraPedido')\
    .sum('Valor')\
    .withColumnRenamed('sum(Valor)', 'Faturamento')\
    .orderBy('Faturamento', ascending= False)\
    .show()

+----------+-----------+
|HoraPedido|Faturamento|
+----------+-----------+
|        14|      53164|
|        13|      53089|
|        10|      52940|
|        18|      52827|
|        11|      52767|
|        16|      52691|
|        20|      52614|
|        17|      52276|
|        15|      52076|
|        19|      52076|
|        12|      51969|
|         9|      51366|
|        21|        815|
+----------+-----------+



##### Faturamento por mês

In [ ]:
pedidos.groupBy(month('Data').alias('Mes'))\
    .sum('Valor')\
    .withColumnRenamed('sum(Valor)', 'Faturamento')\
    .orderBy('Faturamento', ascending= False)\
    .show()

+---+-----------+
|Mes|Faturamento|
+---+-----------+
|  3|      57079|
|  7|      55035|
|  5|      54605|
| 10|      54513|
| 12|      54092|
|  1|      51994|
|  8|      51842|
| 11|      51754|
|  6|      51415|
|  9|      50831|
|  4|      49627|
|  2|      47883|
+---+-----------+



##### Faturamento por ano

In [ ]:
pedidos.groupBy(year('Data').alias('Ano'))\
    .sum('Valor')\
    .withColumnRenamed('sum(Valor)', 'Faturamento')\
    .orderBy('Faturamento', ascending= False)\
    .show()

+----+-----------+
| Ano|Faturamento|
+----+-----------+
|2022|     158640|
|2023|     157910|
|2021|     157075|
|2020|     157045|
+----+-----------+



##### Faturamento por dia do mes

In [ ]:
pedidos.groupBy(dayofmonth('Data').alias('Dia'))\
    .sum('Valor')\
    .withColumnRenamed('sum(Valor)', 'Faturamento')\
    .orderBy('Faturamento', ascending= False)\
    .show()

+---+-----------+
|Dia|Faturamento|
+---+-----------+
|  1|      28601|
|  2|      21770|
|  3|      21673|
|  8|      21631|
| 25|      21154|
| 18|      21104|
| 17|      20915|
| 12|      20842|
| 21|      20701|
|  5|      20526|
| 11|      20486|
| 15|      20429|
| 26|      20417|
| 13|      20407|
| 14|      20289|
| 16|      20228|
|  6|      20195|
| 20|      20167|
| 23|      20144|
| 27|      20143|
+---+-----------+
only showing top 20 rows



##### Faturamento por cidade

In [ ]:
pedidos.groupBy('Cidade')\
    .sum('Valor')\
    .withColumnRenamed('sum(Valor)', 'Faturamento')\
    .orderBy('Faturamento', ascending= False)\
    .show()

+-------------------+-----------+
|             Cidade|Faturamento|
+-------------------+-----------+
|           Blumenau|      38160|
|         Petrópolis|      38046|
|     Rio de Janeiro|      37854|
|             Itajaí|      37658|
|          Fortaleza|      37319|
|São José dos Campos|      37279|
|      Florianópolis|      37216|
|          Joinville|      37194|
|        Nova Iguaçu|      37039|
|             Sobral|      36976|
|         Jaguariúna|      36952|
|          Nilópolis|      36816|
|  Juazeiro do Norte|      36747|
|    Duque de Caxias|      36654|
|           Caieiras|      36646|
|   Campos do Jordão|      36142|
|          São Paulo|      35972|
+-------------------+-----------+



##### Faturamento por estado

In [ ]:
pedidos.groupBy('Estado')\
    .sum('Valor')\
    .withColumnRenamed('sum(Valor)', 'Faturamento')\
    .orderBy('Faturamento', ascending= False)\
    .show()

+--------------+-----------+
|        Estado|Faturamento|
+--------------+-----------+
|Rio de Janeiro|     186409|
|     São Paulo|     182991|
|Santa Catarina|     150228|
|         Ceará|     111042|
+--------------+-----------+



##### Faturamento por região

In [ ]:
pedidos.groupBy('Regiao')\
    .sum('Valor')\
    .withColumnRenamed('sum(Valor)', 'Faturamento')\
    .orderBy('Faturamento', ascending= False)\
    .show()

+--------+-----------+
|  Regiao|Faturamento|
+--------+-----------+
| Sudeste|     369400|
|     Sul|     150228|
|Nordeste|     111042|
+--------+-----------+



##### Faturamento por consumo (entrega ou no local)

In [ ]:
pedidos.groupBy('TipoEntrega')\
    .sum('Valor')\
    .withColumnRenamed('sum(Valor)', 'Faturamento')\
    .orderBy('Faturamento', ascending= False)\
    .show()

+----------------+-----------+
|     TipoEntrega|Faturamento|
+----------------+-----------+
|Consumo no local|     316137|
|        Delivery|     314533|
+----------------+-----------+



##### Faturamento por tipo de pagamento

In [ ]:
pedidos.groupBy('TipoPagamento')\
    .sum('Valor')\
    .withColumnRenamed('sum(Valor)', 'Faturamento')\
    .orderBy('Faturamento', ascending= False)\
    .show()

+-------------+-----------+
|TipoPagamento|Faturamento|
+-------------+-----------+
|     Dinheiro|     158915|
|       Débito|     158430|
|      Crédito|     156721|
|          Pix|     156604|
+-------------+-----------+



##### Faturamento por mês onde o consumo foi no local

In [ ]:
pedidos.groupBy(month('Data').alias('Mes'), 'TipoEntrega')\
    .sum('Valor')\
    .withColumnRenamed('sum(Valor)', 'Faturamento')\
    .filter(col('TipoEntrega') == 'Consumo no local')\
    .orderBy('Faturamento', ascending= False)\
    .show()

+---+----------------+-----------+
|Mes|     TipoEntrega|Faturamento|
+---+----------------+-----------+
|  3|Consumo no local|      28707|
|  7|Consumo no local|      28109|
|  5|Consumo no local|      27872|
| 10|Consumo no local|      27401|
| 12|Consumo no local|      27285|
|  9|Consumo no local|      26199|
|  1|Consumo no local|      25817|
|  8|Consumo no local|      25686|
| 11|Consumo no local|      25435|
|  6|Consumo no local|      25038|
|  4|Consumo no local|      24775|
|  2|Consumo no local|      23813|
+---+----------------+-----------+



##### Faturamento por mês onde o consumo foi por delivery

In [ ]:
pedidos.groupBy(month('Data').alias('Mes'), 'TipoEntrega')\
    .sum('Valor')\
    .withColumnRenamed('sum(Valor)', 'Faturamento')\
    .filter(col('TipoEntrega') == 'Delivery')\
    .orderBy('Faturamento', ascending= False)\
    .show()

+---+-----------+-----------+
|Mes|TipoEntrega|Faturamento|
+---+-----------+-----------+
|  3|   Delivery|      28372|
| 10|   Delivery|      27112|
|  7|   Delivery|      26926|
| 12|   Delivery|      26807|
|  5|   Delivery|      26733|
|  6|   Delivery|      26377|
| 11|   Delivery|      26319|
|  1|   Delivery|      26177|
|  8|   Delivery|      26156|
|  4|   Delivery|      24852|
|  9|   Delivery|      24632|
|  2|   Delivery|      24070|
+---+-----------+-----------+



##### Faturamento por mes por tipo de pagamento

In [ ]:
pedidos.groupBy(month('Data').alias('Mes'), 'TipoPagamento')\
    .sum('Valor')\
    .withColumnRenamed('sum(Valor)', 'Faturamento')\
    .orderBy('Faturamento', ascending= False)\
    .show()

+---+-------------+-----------+
|Mes|TipoPagamento|Faturamento|
+---+-------------+-----------+
|  3|      Crédito|      14313|
|  3|     Dinheiro|      14298|
|  3|          Pix|      14271|
|  5|     Dinheiro|      14198|
|  3|       Débito|      14197|
| 10|      Crédito|      14061|
|  7|       Débito|      13879|
|  5|          Pix|      13871|
| 12|     Dinheiro|      13833|
|  7|          Pix|      13804|
|  7|      Crédito|      13769|
| 10|     Dinheiro|      13756|
| 12|       Débito|      13753|
| 10|       Débito|      13687|
|  1|     Dinheiro|      13632|
|  7|     Dinheiro|      13583|
|  9|          Pix|      13333|
| 12|          Pix|      13300|
|  5|      Crédito|      13278|
|  5|       Débito|      13258|
+---+-------------+-----------+
only showing top 20 rows



##### Faturamento por dia do mes mes por tipo de pagamento

In [ ]:
pedidos.groupBy(dayofmonth('Data').alias('Dia'), 'TipoPagamento')\
    .sum('Valor')\
    .withColumnRenamed('sum(Valor)', 'Faturamento')\
    .orderBy('Faturamento', ascending= False)\
    .show()

+---+-------------+-----------+
|Dia|TipoPagamento|Faturamento|
+---+-------------+-----------+
|  1|       Débito|       7276|
|  1|     Dinheiro|       7192|
|  1|      Crédito|       7132|
|  1|          Pix|       7001|
|  3|       Débito|       5799|
| 12|       Débito|       5733|
|  2|      Crédito|       5700|
|  8|      Crédito|       5561|
| 25|      Crédito|       5510|
| 18|          Pix|       5490|
|  9|     Dinheiro|       5488|
| 21|     Dinheiro|       5471|
|  8|     Dinheiro|       5452|
| 13|       Débito|       5448|
|  2|     Dinheiro|       5441|
|  2|       Débito|       5410|
|  3|      Crédito|       5403|
|  8|       Débito|       5399|
| 23|       Débito|       5391|
| 15|          Pix|       5376|
+---+-------------+-----------+
only showing top 20 rows

